In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line9.csv')
data
# 카페 데이터만
data = data.loc[data['상권업종중분류명'] == '커피점/카페']  
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
7,66,28524822,빈트리망원점,망원점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,121230,4007.0,NaN,1,NaN,126.899339,37.556041,9호선 선유도,1294.649872,NaN
13,137,15520570,더블유카페,역삼2호점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135921,6225.0,NaN,1,2,127.043902,37.499426,9호선 선정릉,764.611819,NaN
15,147,22646116,커피빈,코리아선릉역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135080,6150.0,NaN,1,NaN,127.048223,37.504750,9호선 삼성중앙,461.237983,NaN
29,222,25437483,카페그리다,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135090,6159.0,NaN,1,NaN,127.052597,37.506767,9호선 삼성중앙,554.451907,NaN
35,274,23739181,홈플러스어라운드홈,강서점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157030,7567.0,NaN,1,NaN,126.854800,37.558456,9호선 가양,236.335955,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12774,135527,17249536,티컬렉티브,삼성점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135868,6097.0,NaN,NaN,NaN,127.048787,37.511981,9호선 삼성중앙,35.645121,NaN
12782,135625,17273431,트윗젤,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135876,6212.0,220,NaN,116,127.048927,37.504494,9호선 삼성중앙,478.709107,NaN
12784,135632,17254243,후르츠13.5,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157805,7788.0,NaN,12,NaN,126.825403,37.568078,9호선 마곡나루,235.823443,NaN
12785,135633,17252972,카페,MJ보드,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157805,7788.0,NaN,13,NaN,126.827682,37.568569,9호선 마곡나루,135.140328,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
7,빈트리망원점,커피점/카페,커피전문점/카페/다방,서울특별시 마포구 망원로 22,9호선 선유도,1294.649872,NaN,선유도,선유도역 빈트리망원점
13,더블유카페,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로 407,9호선 선정릉,764.611819,NaN,선정릉,선정릉역 더블유카페
15,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로 335,9호선 삼성중앙,461.237983,NaN,삼성중앙,삼성중앙역 커피빈
29,카페그리다,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로69길 14,9호선 삼성중앙,554.451907,NaN,삼성중앙,삼성중앙역 카페그리다
35,홈플러스어라운드홈,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 화곡로 398,9호선 가양,236.335955,NaN,가양,가양역 홈플러스어라운드홈
...,...,...,...,...,...,...,...,...,...
12774,티컬렉티브,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 봉은사로 449,9호선 삼성중앙,35.645121,NaN,삼성중앙,삼성중앙역 티컬렉티브
12782,트윗젤,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로 340,9호선 삼성중앙,478.709107,NaN,삼성중앙,삼성중앙역 트윗젤
12784,후르츠13.5,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 마곡서로 158,9호선 마곡나루,235.823443,NaN,마곡나루,마곡나루역 후르츠13.5
12785,카페,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 마곡중앙로 165,9호선 마곡나루,135.140328,NaN,마곡나루,마곡나루역 카페


In [4]:
items = data["검색어"]
items

7          선유도역 빈트리망원점
13          선정릉역 더블유카페
15           삼성중앙역 커피빈
29         삼성중앙역 카페그리다
35       가양역 홈플러스어라운드홈
             ...      
12774      삼성중앙역 티컬렉티브
12782        삼성중앙역 트윗젤
12784    마곡나루역 후르츠13.5
12785         마곡나루역 카페
12799        노량진역 카페시모
Name: 검색어, Length: 2215, dtype: object

In [5]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [6]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/2215
식당 존재 x
진행상황 : 2/2215
식당 존재
https://place.map.kakao.com/22646026
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/2215
식당 존재
https://place.map.kakao.com/17101536
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/2215
식당 존재
https://place.map.kakao.com/321704063
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 5/2215
식당 존재 x
진행상황 : 6/2215
식당 존재
https://place.map.kakao.com/21405265
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/2215
식당 존재
https://place.map.kakao.com/1636263278
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 8/2215
식당 존재 x
진행상황 : 9/2215
식당 존재 x
진행상황 : 10/2215
식당 존재 x
진행상황 : 11/2215
식당 존재
https://place.map.kakao.com/639700545
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/2215
식당 존재
https://place.map.kakao.com/1522483669
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/2215
식당 존재
https://place.map.kakao.com/860472511
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 14/2215
식당 존재 x
진행상황 : 15/2215
식당 존재
https://place.map.kakao.com/24765625
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 16/2215
식당 존재
https://place.map.kakao.com/1862570607
0
리

5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 134/2215
식당 존재 x
진행상황 : 135/2215
식당 존재 x
진행상황 : 136/2215
식당 존재
https://place.map.kakao.com/1529498058
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 137/2215
식당 존재
https://place.map.kakao.com/8438452
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 138/2215
식당 존재 x
진행상황 : 139/2215
식당 존재
https://place.map.kakao.com/21532642
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 140/2215
식당 존재 x
진행상황 : 141/2215
식당 존재
https://place.map.kakao.com/440142287
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 142/2215
식당 존재 x
진행상황 : 143/2215
식당 존재
https://place.map.kakao.com/14712077
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 144/2215
식당 존재
https://place.map.kakao.com/668868626
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 145/2215
식당 존재
https://place.map.kakao.com/10555584
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 146/2215
식당 존재
https://place.map.kakao.com/1588964309
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 147/2215
식당 존재
https://place.map.kakao.com/1943853072
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 148/2215
식당 존재 x
진행상황 : 149/2215
식당 존

식당 존재 x
진행상황 : 260/2215
식당 존재
https://place.map.kakao.com/555820689
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 261/2215
식당 존재 x
진행상황 : 262/2215
식당 존재 x
진행상황 : 263/2215
식당 존재
https://place.map.kakao.com/1612787942
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 264/2215
식당 존재
https://place.map.kakao.com/49050456
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 265/2215
식당 존재
https://place.map.kakao.com/26338066
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 266/2215
식당 존재 x
진행상황 : 267/2215
식당 존재
https://place.map.kakao.com/13057629
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 268/2215
식당 존재 x
진행상황 : 269/2215
식당 존재
https://place.map.kakao.com/1492157077
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 270/2215
식당 존재
https://place.map.kakao.com/15856424
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 271/2215
식당 존재
https://place.map.kakao.com/27393443
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 272/2215
식당 존재
https://place.map.kakao.com/1909394228
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 273/2215
식당 존재 x
진행상황 : 274/2215
식당 존재 x
진행상황 : 275/22

식당 존재
https://place.map.kakao.com/19634852
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 375/2215
식당 존재 x
진행상황 : 376/2215
식당 존재
https://place.map.kakao.com/12969107
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 377/2215
식당 존재
https://place.map.kakao.com/1992321980
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 378/2215
식당 존재 x
진행상황 : 379/2215
식당 존재 x
진행상황 : 380/2215
식당 존재 x
진행상황 : 381/2215
식당 존재 x
진행상황 : 382/2215
식당 존재 x
진행상황 : 383/2215
식당 존재
https://place.map.kakao.com/1442382250
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 384/2215
식당 존재
https://place.map.kakao.com/1306584702
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 385/2215
식당 존재
https://place.map.kakao.com/21544362
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 386/2215
식당 존재
https://place.map.kakao.com/896805702
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 387/2215
식당 존재
https://place.map.kakao.com/60371772
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 388/2215
식당 존재
https://place.map.kakao.com/245586838
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 389/2215
식당 존재
https://place.map.kakao.com/1170

식당 존재
https://place.map.kakao.com/24529846
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 496/2215
식당 존재 x
진행상황 : 497/2215
식당 존재
https://place.map.kakao.com/1897472440
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 498/2215
식당 존재
https://place.map.kakao.com/27544715
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 499/2215
식당 존재 x
진행상황 : 500/2215
식당 존재 x
진행상황 : 501/2215
식당 존재
https://place.map.kakao.com/828172750
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 502/2215
식당 존재
https://place.map.kakao.com/27579564
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 503/2215
식당 존재
https://place.map.kakao.com/10751106
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 504/2215
식당 존재
https://place.map.kakao.com/13322805
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 505/2215
식당 존재
https://place.map.kakao.com/555820689
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 506/2215
식당 존재 x
진행상황 : 507/2215
식당 존재
https://place.map.kakao.com/2053156274
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 508/2215
식당 존재 x
진행상황 : 509/2215
식당 존재
https://place.map.kakao.com/246068031
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진

...

식당 존재 x
진행상황 : 874/2215
식당 존재 x
진행상황 : 875/2215
식당 존재
https://place.map.kakao.com/1344477447
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 876/2215
식당 존재
https://place.map.kakao.com/851605449
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 877/2215
식당 존재
https://place.map.kakao.com/177236465
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 878/2215
식당 존재
https://place.map.kakao.com/21544362
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 879/2215
식당 존재
https://place.map.kakao.com/13085894
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 880/2215
식당 존재
https://place.map.kakao.com/1336926358
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 881/2215
식당 존재
https://place.map.kakao.com/176158301
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 882/2215
식당 존재 x
진행상황 : 883/2215
식당 존재
https://place.map.kakao.com/301579321
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 884/2215
식당 존재
https://place.map.kakao.com/1590155080
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 885/2215
식당 존재
https://place.map.kakao.com/1506614333
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 886/2215
식당 존재
http

식당 존재
https://place.map.kakao.com/27114008
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 992/2215
식당 존재
https://place.map.kakao.com/24583134
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 993/2215
식당 존재 x
진행상황 : 994/2215
식당 존재 x
진행상황 : 995/2215
식당 존재
https://place.map.kakao.com/365973809
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 996/2215
식당 존재 x
진행상황 : 997/2215
식당 존재
https://place.map.kakao.com/310929360
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 998/2215
식당 존재
https://place.map.kakao.com/8520953
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 999/2215
식당 존재
https://place.map.kakao.com/14977155
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1000/2215
식당 존재 x
진행상황 : 1001/2215
식당 존재 x
진행상황 : 1002/2215
식당 존재
https://place.map.kakao.com/1079886042
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1003/2215
식당 존재
https://place.map.kakao.com/1986565753
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1004/2215
식당 존재
https://place.map.kakao.com/89072721
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1005/2215
식당 존재
https://place.map.kakao.com/1561245373
2
더 이상 리뷰 존재 X
기본 페이지로 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-a363dfa393a7>", line 39, in <module>
    time.sleep(20)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1170, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line

TypeError: object of type 'NoneType' has no len()

In [7]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,None,선정릉역 더블유카페,None,2018.03.25.
1,mmra8v,삼성중앙역 커피빈,3점,2020.06.23.
2,b63cqb,삼성중앙역 커피빈,5점,2019.07.30.
3,8uftua,삼성중앙역 커피빈,5점,2018.08.15.
4,None,삼성중앙역 커피빈,None,2017.09.07.
...,...,...,...,...
3649,e1qio8,양천향교역 노르웨이숲,1점,2018.12.23.
3650,h6mqu3,양천향교역 노르웨이숲,5점,2018.07.14.
3651,None,양천향교역 노르웨이숲,None,2018.04.23.
3652,,양천향교역 노르웨이숲,5점,2017.05.11.


In [8]:
img_elem

,ItemID,img_url
0,선정릉역 더블유카페,None
1,삼성중앙역 커피빈,None
2,삼성중앙역 커피빈,None
3,삼성중앙역 커피빈,None
4,삼성중앙역 커피빈,None
...,...,...
3649,양천향교역 노르웨이숲,None
3650,양천향교역 노르웨이숲,None
3651,양천향교역 노르웨이숲,None
3652,양천향교역 노르웨이숲,None


In [9]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
7,빈트리망원점,커피점/카페,커피전문점/카페/다방,서울특별시 마포구 망원로 22,9호선 선유도,1294.649872,NaN,선유도,선유도역 빈트리망원점
13,더블유카페,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로 407,9호선 선정릉,764.611819,https://place.map.kakao.com/22646026,선정릉,선정릉역 더블유카페
15,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로 335,9호선 삼성중앙,461.237983,https://place.map.kakao.com/17101536,삼성중앙,삼성중앙역 커피빈
29,카페그리다,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로69길 14,9호선 삼성중앙,554.451907,https://place.map.kakao.com/321704063,삼성중앙,삼성중앙역 카페그리다
35,홈플러스어라운드홈,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 화곡로 398,9호선 가양,236.335955,None,가양,가양역 홈플러스어라운드홈
...,...,...,...,...,...,...,...,...,...
12774,티컬렉티브,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 봉은사로 449,9호선 삼성중앙,35.645121,NaN,삼성중앙,삼성중앙역 티컬렉티브
12782,트윗젤,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로 340,9호선 삼성중앙,478.709107,NaN,삼성중앙,삼성중앙역 트윗젤
12784,후르츠13.5,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 마곡서로 158,9호선 마곡나루,235.823443,NaN,마곡나루,마곡나루역 후르츠13.5
12785,카페,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 마곡중앙로 165,9호선 마곡나루,135.140328,NaN,마곡나루,마곡나루역 카페


In [10]:
img_elem

,ItemID,img_url
0,선정릉역 더블유카페,None
1,삼성중앙역 커피빈,None
2,삼성중앙역 커피빈,None
3,삼성중앙역 커피빈,None
4,삼성중앙역 커피빈,None
...,...,...
3649,양천향교역 노르웨이숲,None
3650,양천향교역 노르웨이숲,None
3651,양천향교역 노르웨이숲,None
3652,양천향교역 노르웨이숲,None


In [11]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,None,선정릉역 더블유카페,None,2018.03.25.
1,mmra8v,삼성중앙역 커피빈,3점,2020.06.23.
2,b63cqb,삼성중앙역 커피빈,5점,2019.07.30.
3,8uftua,삼성중앙역 커피빈,5점,2018.08.15.
4,None,삼성중앙역 커피빈,None,2017.09.07.
...,...,...,...,...
3649,e1qio8,양천향교역 노르웨이숲,1점,2018.12.23.
3650,h6mqu3,양천향교역 노르웨이숲,5점,2018.07.14.
3651,None,양천향교역 노르웨이숲,None,2018.04.23.
3652,,양천향교역 노르웨이숲,5점,2017.05.11.


In [12]:
review_elem

,ItemID,review
0,선정릉역 더블유카페,보통이에요
1,삼성중앙역 커피빈,무난했어요
2,삼성중앙역 커피빈,최고예요
3,삼성중앙역 커피빈,친절합니다ㅎ ㅎ
4,삼성중앙역 커피빈,좋아요
...,...,...
3649,양천향교역 노르웨이숲,LP보유량도 부족하고 LP판 보다 라이브 동영상을 트는 경우가 더 많습니다. 라이브...
3650,양천향교역 노르웨이숲,추억을 느낄수있고 여유와 낭만이 있는 단체모임하기 좋은곳입니다
3651,양천향교역 노르웨이숲,LP 판을 듣는 곳
3652,양천향교역 노르웨이숲,분위기가 너무 좋아요 저는 김광석 팬이라 신청곡을 그날들 부탁드렸는데요 LP...


In [13]:
img_elem.head()

,ItemID,img_url
0,선정릉역 더블유카페,None
1,삼성중앙역 커피빈,None
2,삼성중앙역 커피빈,None
3,삼성중앙역 커피빈,None
4,삼성중앙역 커피빈,None


In [16]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




      UserID       ItemID Rating    Timestamp
3649  e1qio8  양천향교역 노르웨이숲     1점  2018.12.23.
3650  h6mqu3  양천향교역 노르웨이숲     5점  2018.07.14.
3651    None  양천향교역 노르웨이숲   None  2018.04.23.
3652          양천향교역 노르웨이숲     5점  2017.05.11.
3653    None  양천향교역 노르웨이숲   None  2017.05.11.
           ItemID img_url
3649  양천향교역 노르웨이숲    None
3650  양천향교역 노르웨이숲    None
3651  양천향교역 노르웨이숲    None
3652  양천향교역 노르웨이숲    None
3653  양천향교역 노르웨이숲    None
           ItemID                                             review
3649  양천향교역 노르웨이숲  LP보유량도 부족하고 LP판 보다 라이브 동영상을 트는 경우가 더 많습니다. 라이브...
3650  양천향교역 노르웨이숲                 추억을 느낄수있고 여유와 낭만이 있는 단체모임하기 좋은곳입니다
3651  양천향교역 노르웨이숲                                         LP 판을 듣는 곳
3652  양천향교역 노르웨이숲  분위기가 너무 좋아요   저는 김광석 팬이라 신청곡을 그날들 부탁드렸는데요   LP...
3653  양천향교역 노르웨이숲  분위기가 너무 좋아요   저는 김광석 팬이라 신청곡을 그날들 부탁드렸는데요   LP...


In [17]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line9_cafe(1025).csv')

In [18]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line9_cafe_User(1025).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line9_cafe_img(1025).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lline9_cafe_review(1025).csv')

In [19]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


422

In [20]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line9(1025)url.csv')